In [ ]:
from chi import server, context, lease
import os

context.version = "1.0" 
context.choose_project()
context.choose_site(default="CHI@TACC")

In [ ]:
l = lease.get_lease(f"") 
l.show()

In [ ]:
username = os.getenv('USER') # all exp resources will have this prefix
s = server.Server(
    f"node-mltrain-{username}", 
    reservation_id=l.node_reservations[0]["id"],
    image_name="CC-Ubuntu24.04-CUDA"
)
s.submit(idempotent=True)

In [ ]:
s.associate_floating_ip()

In [ ]:
s.refresh()
s.check_connectivity()

In [ ]:
s.refresh()
s.show(type="widget")

In [ ]:
s.execute("curl -sSL https://get.docker.com/ | sudo sh")
s.execute("sudo groupadd -f docker; sudo usermod -aG docker $USER")
s.execute("docker run hello-world")

In [ ]:
s.execute("curl -fsSL https://nvidia.github.io/libnvidia-container/gpgkey | sudo gpg --dearmor -o /usr/share/keyrings/nvidia-container-toolkit-keyring.gpg \
  && curl -s -L https://nvidia.github.io/libnvidia-container/stable/deb/nvidia-container-toolkit.list | \
    sed 's#deb https://#deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://#g' | \
    sudo tee /etc/apt/sources.list.d/nvidia-container-toolkit.list")
s.execute("sudo apt update")
s.execute("sudo apt-get install -y nvidia-container-toolkit")
s.execute("sudo nvidia-ctk runtime configure --runtime=docker")
# for https://github.com/NVIDIA/nvidia-container-toolkit/issues/48
s.execute("sudo jq 'if has(\"exec-opts\") then . else . + {\"exec-opts\": [\"native.cgroupdriver=cgroupfs\"]} end' /etc/docker/daemon.json | sudo tee /etc/docker/daemon.json.tmp > /dev/null && sudo mv /etc/docker/daemon.json.tmp /etc/docker/daemon.json")
s.execute("sudo systemctl restart docker")

In [ ]:
s.execute("docker run --rm --gpus all ubuntu nvidia-smi")

In [ ]:
s.execute("sudo apt update")
s.execute("sudo apt -y install nvtop")

In [ ]:
s.execute("git clone https://github.com/RaghuHemadri/mlops_test_raghu.git")

In [ ]:
s.execute("export YOUR_USER_ID='d25b59d03a410639b308fc4180ae27bbdcb3eaafc746715255092b23166344ae'; export APP_CRED_ID='b927f3996058472a81c13c5f2f34d2e7'; export APP_CRED_SECRET='GiGd8lzNvcJ-m7qeZCU-5oQqDZtyhNrlMtlLdmGU07WRe76AxPDyglTj_ZKpS9P6csx3HSnDLFPQMP307QJxuw'; cd mlops_test_raghu; bash run_etl.sh")

In [ ]:
s.execute("export HOST_IP=$(curl --silent http://169.254.169.254/latest/meta-data/public-ipv4); cd mlops_test_raghu; docker compose -f docker-compose-ray.yaml up --build -d")
s.execute("docker ps")

In [ ]:
s.execute("cd mlops_test_raghu; sudo cp train_ray.py workspace/train_ray.py; sudo cp ray_requirements.txt workspace/ray_requirements.txt; sudo cp ray_runtime.json workspace/ray_runtime.json")

In [ ]:
s.execute("docker logs jupyter-ray")